In [1]:
import pandas as pd
import ipaddress
import json
import os
from fastFlow.flowprintOptimal.sekigo.flowUtils.commons import getIATFromTimeStamps, saveFlows
from fastFlow.flowprintOptimal.sekigo.core.flowRepresentation import MediaStreamRepresentation


In [2]:
def getIpToProv(file_path = "data/conf_prov_to_ips.json"):
    with open(file_path, "r") as f:
        prov_to_ips = json.loads(f.read())
    
    ip_to_prov = dict()
    for prov,ips in prov_to_ips.items():

        for ip in ips:
            if ip in ip_to_prov:
                print("Fllo")
                continue
            ip_to_prov[ip] = prov
    return ip_to_prov


def getDfFromCSV(csv_path, timestamp_columns = []):
    def formatDateTimeString(string):
        string =  " ".join(string.split(" ")[:2])

        # now in case the time is not in the format of 2021-01-01 00:00:00.000000 we add the .000000
        if len(string.split(" ")[1].split(".")) == 1:
            string += ".000000"
        return string
        
    df = pd.read_csv(csv_path).dropna()
    for timestamp_column in timestamp_columns:

        
        df[f"{timestamp_column}"] = df[f"{timestamp_column}"].apply(formatDateTimeString)
        df[f"{timestamp_column}"] = pd.to_datetime(df[f"{timestamp_column}"])

    
    return df

def processFiveTuple(df,ip_to_prov_path):
    ip_to_prov = getIpToProv(file_path= ip_to_prov_path)

    src_ips, src_ports , dst_ips, dst_ports, protocols = [],[],[],[],[]
    provs = []

    for i in range(len(df)):
        s = df.iloc[i].FiveTuple
        src_addr, dst_addr, protocol = s.split("->")
        src_ip,src_port = src_addr.split(":")
        dst_ip,dst_port = dst_addr.split(":")
        protocol = int(protocol.split(":")[1][:-1].strip())

        src_ips.append(src_ip)
        src_ports.append(src_port)
        dst_ips.append(dst_ip)
        dst_ports.append(dst_port)
        protocols.append(protocol)

        if src_ip in ip_to_prov:
            provs.append(ip_to_prov[src_ip])
        elif dst_ip in ip_to_prov:
            provs.append(ip_to_prov[dst_ip])
        else:
            provs.append("UNK")
    
    df["src_ip"] = src_ips
    df["src_port"] = src_ports
    df["dst_ip"] = dst_ips
    df["dst_port"] = dst_ports
    df["protocol"] = protocols
    df["provider"] = provs

    df["src_unsw"] = df["src_ip"].apply(lambda ip : ip.startswith("129.94"))
    df["dst_unsw"] = df["dst_ip"].apply(lambda ip :ip.startswith("129.94"))

    

    df.drop(columns= ["FiveTuple"], inplace= True)
    return df

def getFlowIDToProv(df):
    flow_id_to_prov = dict()
    for i in range(len(df)):
        row = df.iloc[i]
        flow_id = row.FlowId
        prov = row.provider

        if flow_id in flow_id_to_prov:
            assert False
        
        flow_id_to_prov[flow_id] = prov
    
    return flow_id_to_prov
    

In [6]:
def extractRepresentationFeatures(df):
    directions, timestamps, lengths, packet_types = [],[],[], []


    for i in range(len(df)):
        row = df.iloc[i]
        directions.append(int(row.Direction))
        timestamps.append(row.Timestamp)
        lengths.append(int(row.PacketLength))

        if row.RTP == True:
            packet_types.append(MediaStreamRepresentation.PacketType.RTP)
        elif row.DTLS == True:
            packet_types.append(MediaStreamRepresentation.PacketType.DTLS)
        elif row.STUN == True:
            packet_types.append(MediaStreamRepresentation.PacketType.STUN)
        else:
            packet_types.append(MediaStreamRepresentation.PacketType.UNK)

    IATs = getIATFromTimeStamps(timestamps)
    return directions,IATs,lengths,packet_types

    

In [7]:
def getFlowReps(packets_df : pd.DataFrame,flow_id_to_prov):
    flow_reps = []
    for flow_id, mini_df in packets_df.groupby(by= ["FlowID"]):
        flow_id = flow_id[0]
        if flow_id not in flow_id_to_prov:
            continue
        prov = flow_id_to_prov[flow_id]
        mini_df.sort_values(by= "Timestamp", inplace= True)
        directions,IATs,lengths,packet_types  = extractRepresentationFeatures(mini_df)
        flow_rep = MediaStreamRepresentation(lengths= lengths, directions= directions,packet_types= packet_types,
                                              inter_arrival_times= IATs,class_type=prov, provider_type= prov)
        flow_reps.append(flow_rep)
    return flow_reps


In [8]:
def getRepresentation(flows_path, packets_path):
    packets_df,flow_df = getDfFromCSV(csv_path= packets_path, timestamp_columns= ["Timestamp"]), getDfFromCSV(csv_path= flows_path, timestamp_columns= ["StartTime", "EndTime"])
    flow_df = processFiveTuple(flow_df, ip_to_prov_path= "data/conf_prov_to_ips.json")
    flow_df  = flow_df[flow_df.provider != "UNK"]
    flow_id_to_prov = getFlowIDToProv(flow_df)
    packet_flow_reps = getFlowReps(packets_df=packets_df ,flow_id_to_prov= flow_id_to_prov)
    return packet_flow_reps

    
    

In [9]:
class DataPath:
    def __init__(self, flows_path, packets_path):
        self.flows_path = flows_path
        self.packets_path = packets_path
        assert os.path.exists(self.flows_path) and os.path.exists(self.packets_path)

In [10]:
data_paths = [DataPath(flows_path= "data/output/flows.csv", packets_path= "data/output/packets.csv"), DataPath(flows_path= "data/output/flows24.csv", packets_path= "data/output/packets24.csv")]

In [11]:
data_paths += [DataPath(flows_path= "data/output/three_prov_flows.csv", packets_path= "data/output/three_prov_packets.csv")]

In [12]:
representations = []
for data_path in data_paths:
    representations += getRepresentation(flows_path= data_path.flows_path, packets_path= data_path.packets_path)

Fllo
Fllo
Fllo
Fllo


/var/folders/j8/tqqbkh4s5193wzs2k9_ncvcr0000gn/T/ipykernel_78555/3534144415.py:25: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path).dropna()


Fllo
Fllo
Fllo
Fllo
Fllo
Fllo
Fllo
Fllo


In [16]:
types = list(map(lambda x: x.class_type, representations))

In [17]:
pd.Series(types).value_counts()

WhatsAppVoice      11655
GoogleMeet          1204
Microsoft Teams      637
Facebook             589
Signal               472
Discord               29
SkypeCall             14
Telegram              10
Name: count, dtype: int64

In [21]:
needed_class_types = ["GoogleMeet", "Microsoft Teams"]
needed_reps = list(filter(lambda x: x.class_type in needed_class_types, representations))

In [22]:
saveFlows("data/dummy/dummy.json",needed_reps)

In [43]:
index = -99
needed_reps[index].class_type, needed_reps[index].packet_types, needed_reps[index].lengths

('Microsoft Teams',
 [<PacketType.UNK: 4>,
  <PacketType.STUN: 3>,
  <PacketType.STUN: 3>,
  <PacketType.UNK: 4>,
  <PacketType.UNK: 4>,
  <PacketType.STUN: 3>,
  <PacketType.STUN: 3>,
  <PacketType.UNK: 4>,
  <PacketType.UNK: 4>,
  <PacketType.UNK: 4>,
  <PacketType.UNK: 4>,
  <PacketType.UNK: 4>,
  <PacketType.UNK: 4>,
  <PacketType.UNK: 4>,
  <PacketType.UNK: 4>,
  <PacketType.UNK: 4>,
  <PacketType.UNK: 4>,
  <PacketType.STUN: 3>,
  <PacketType.UNK: 4>,
  <PacketType.STUN: 3>,
  <PacketType.UNK: 4>,
  <PacketType.STUN: 3>,
  <PacketType.UNK: 4>,
  <PacketType.UNK: 4>,
  <PacketType.UNK: 4>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <PacketType.RTP: 1>,
  <Pack

In [ ]:
needed_reps[120].class_type